In [1]:
import pandas as pd
train_file = "train.data.txt"
dev_file = "dev.data.txt"
test_file = "test.data.txt"


train_data = pd.read_csv('./%s.csv'%train_file,keep_default_na=False)
dev_data = pd.read_csv('./%s.csv'%dev_file,keep_default_na=False)
test_data = pd.read_csv('./%s.csv'%test_file,keep_default_na=False)


In [ ]:
from transformers import BertTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import numpy as np
import json
from sklearn.preprocessing import LabelEncoder
import re
import scipy
import os
import keras
import string
import tensorflow as tf
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

le = LabelEncoder()
vectorizer = TfidfVectorizer()
def tokenize_tweet(string_data:str):
    stem = PorterStemmer()
    wordnet_lemmatizer = WordNetLemmatizer()
    tokenized = nltk.RegexpTokenizer('\w+')
    data = string_data.replace('\n', '')
    data = data.lower()
    data = re.sub('https?://\S+|www\.\S+', '', data)
    data = re.sub('[%s]' % re.escape(string.punctuation), '', data)
    # data = re.sub('\w*\d\w*', '', data)

    data = tokenized.tokenize(data)
    data = [i for i in data if i not in stopwords]
    data = [stem.stem(word) for word in data]
    data = [wordnet_lemmatizer.lemmatize(word) for word in data]
    data = ' '.join(data)   
    return data
    
def preprocess_token(df, dropNa=True):
    data = df.copy()
    data['main_tweet'] = data['main_tweet'].fillna('')
    if dropNa:
        data.replace('', np.nan, inplace=True)
        data.dropna(subset=['main_tweet'], inplace=True)
    data['main_tweet'] = data['main_tweet'].apply(lambda x: tokenize_tweet(x))
    data['verified'] = data['verified'].astype(int)
    return data
"""
    Preprocessing the training data
"""

train = preprocess_token(train_data)
traintfidf = vectorizer.fit_transform(train['main_tweet'].tolist())
traintfidf = scipy.sparse.csr_matrix(traintfidf).todense()
d = pd.DataFrame(traintfidf)
verified = pd.DataFrame({"verified":train['verified'].tolist()})
followers = pd.DataFrame({"followers":train['followers'].tolist()})
train_label = le.fit_transform(train['label'].tolist())
train = pd.concat([d, verified], axis=1)
train = tf.convert_to_tensor(train)

"""
    Preprocessing the dev data
"""
dev = preprocess_token(dev_data)
devtfidf = vectorizer.transform(dev['main_tweet'].tolist())
devtfidf = scipy.sparse.csr_matrix(devtfidf).todense()
devpd = pd.DataFrame(devtfidf)
dev_label=le.transform(dev['label'].tolist())
dev_verified = pd.DataFrame({"verified":dev['verified'].tolist()})
dev_followers = pd.DataFrame({"followers":dev['followers'].tolist()})
dev = pd.concat([devpd, dev_verified], axis=1)
dev = tf.convert_to_tensor(dev)


"""
    Preprocessing the test data
"""
test = preprocess_token(test_data, dropNa=False)
testtfidf = vectorizer.transform(test['main_tweet'].tolist())
testtfidf = scipy.sparse.csr_matrix(testtfidf).todense()
testpd = pd.DataFrame(testtfidf)
test_verified = pd.DataFrame({"verified":test['verified'].tolist()})
test_followers = pd.DataFrame({"followers":test['followers'].tolist()})
test = pd.concat([testpd, test_verified], axis=1)
test = tf.convert_to_tensor(test)


In [ ]:
dev_data

In [ ]:
dev

In [ ]:
devx = preprocess_token(dev_data,dropNa=False)
devtfidfx = vectorizer.transform(devx['main_tweet'].tolist())
devtfidfx = scipy.sparse.csr_matrix(devtfidfx).todense()
devpdx = pd.DataFrame(devtfidfx)
dev_labelx=le.transform(devx['label'].tolist())
dev_verifiedx = pd.DataFrame({"verified":devx['verified'].tolist()})
dev_followersx = pd.DataFrame({"followers":devx['followers'].tolist()})
devx = pd.concat([devpdx, dev_verifiedx], axis=1)
devx = tf.convert_to_tensor(devx)


In [ ]:
devx

In [ ]:
tokenize_tweet("Does the new coronavirus affect older people, or are younger people also susceptible? #Covid_19 #health https://t.co/Kg0Qb5oMs8")

In [ ]:

from tensorflow.keras import layers
model = tf.keras.Sequential([
    layers.Dense(128, activation='relu',input_shape=(train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

history = model.fit(train, train_label,epochs=20, validation_data=(dev, dev_label) ,batch_size=10)

In [ ]:
prediction = model.predict(test)
prediction = (prediction > 0.5).astype("int32")
prediction = np.ndarray.flatten(prediction)
pd.DataFrame({"Predicted":  prediction}).to_csv('TFIDFNNVerified.csv', index_label="Id")

In [ ]:
# prediction = model.predict(devx)
# prediction = (prediction > 0.5).astype("int32")
# prediction = np.ndarray.flatten(prediction)
# listo = []
# for predict in prediction:
#     if predict == 0:
#         listo.append("nonrumor")
#     else:
#         listo.append("rumor")
# newlisto = pd.DataFrame(listo)
# newlisto
# pd.DataFrame({"Predicted":  listo}).to_csv('TFIDFNNVerified.csv', header=False, index = False)

In [ ]:
from sklearn.metrics import classification_report
predictionx = model.predict(dev)
predictionx = (predictionx > 0.5).astype("int32")
predictionx = np.ndarray.flatten(predictionx)
print("TFIDFNNVerified on dev set")
print(classification_report(dev_label, predictionx, digits=4))

In [ ]:
# import os


# def update_data(df, filename):
#     copy_test = df.copy()
#     data_empty_test = copy_test[copy_test['main_tweet'].isna()]
#     listid = data_empty_test['main_tweet_id'].tolist()
#     print(len(listid))
#     i = 0
#     for id in listid:
#         file = './project-data/tweet-objects/tweet-objects/%s.json'%id
#         if os.path.exists(file):
#             with open(file, 'r') as json_file:
#                 data = json.load(json_file)
#                 copy_test.loc[copy_test['main_tweet_id'] == id, 'verified'] = data['user']['verified']
#                 copy_test.loc[copy_test['main_tweet_id'] == id, 'followers'] = data['user']['followers_count']
#                 copy_test.loc[copy_test['main_tweet_id'] == id, 'main_tweet'] = data['text']
#                 i+=1

#     copy_test.to_csv(filename, index=False)

    

In [ ]:

# t2 = pd.read_csv('train.data.txt.csv')
# t3 = pd.read_csv('dev.data.txt.csv')
# t4 = pd.read_csv('test.data.txt.csv')
# update_data(t2, 'train.data.txt.csv')
# update_data(t3, 'dev.data.txt.csv')
# update_data(t4, 'test.data.txt.csv')
